TODO
- batching
- general speed up
- fix overlapping_tfs function 

In [ ]:
import os
import sys

# Dependency imports

from tensor2tensor.bin import t2t_trainer
from tensor2tensor.data_generators import text_encoder
from tensor2tensor.utils import decoding
from tensor2tensor.utils import registry
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import usr_dir

import tensorflow as tf

# for metrics
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

sys.path.append("../tfti")
import tfti



from itertools import combinations
import math
import bisect
import sys
from skpp import ProjectionPursuitRegressor
import numpy as np
import random

sys.path.append("../tfti")
import tfti_infer

sys.path.append("../shapley")
import shapley

import time

# Testing MSA

First, run the real Shapley values. This is an example taken from the Wiki https://en.wikipedia.org/wiki/Shapley_value, demonstrating the contributions of workers (w) and bosses (o), where the value function is mp if the boss o is in the set, and 0 otherwise, where m = the number of workers in the set S and p is the profit from each worker.


First, we compute the full Shapley values.

## Full Shapley Computation

In [ ]:
from importlib import reload
reload(shapley)
p = 1 # contribution of each worker

# 4 workers and 1 owner
players = np.array(['w','w','w','w','o'])
k = (players == 'w').sum()

def v(S):
    if ('o' in S):
        m = (S == 'w').sum()
        return m * p
    else:
        return 0

orderedList = list([i for i in range(len(players))])
R_orderings = shapley.power_set(orderedList)    
characteristic_function = list(map(lambda x: v(players[x]), R_orderings))

# compute the actual Shapley value
shapleys = shapley.compute_shapley_values(players, characteristic_function)
print("Shapley values for all workers are %s" % shapleys)

### Analyzing the reuslts

As you can see, all workers have a contribution of 0.5, and the boss ('o') has a contribution of 2.

In [ ]:
print("Had to compute %i instances of the value function " % (len(R_orderings)))
print(shapleys)
assert(shapleys[0] == p/2)
assert(shapleys[4] == (k * p)/2)

## Computing the Predicted Shapley Values

Next, we compute the Shapley values on predicted value functions using the MSA algorithm. The main idea is that we do not need to compute all 31 instances of the value function. Instead, we will only compute 20 instances and use these to predict the missing value functions. This is useful because in our case, the value function is prediction the model and we have many TFs to predict on.

In [ ]:
# Create all possible orderings of the players
orderedList = list([i for i in range(len(players))])
R_orderings = list(enumerate(shapley.power_set(orderedList)))

# choose random subset of R_orderings
sample_size = 20
R_sampled_orderings = random.sample(R_orderings, sample_size)
characteristic_function = np.zeros(len(R_orderings))

# compute the characteristic function for sampled orderings
# in our case, this will be computed from evaluating our model, building a mask from the current ordering
train_data = np.zeros((sample_size, len(players)))
y = np.zeros(sample_size)

for i, x in enumerate(R_sampled_orderings):
    y[i] = v(players[x[1]])
    for j in x[1]:
        train_data[i,j] = 1
    characteristic_function[x[0]] = y[i] 
    

# build a matrix that is #random calcs by len(players)  and predict on v
estimator = ProjectionPursuitRegressor()
estimator.fit(train_data, y)

# predict the missing v(s) 
missing_R_orderings = [x for x in R_orderings if x not in R_sampled_orderings]
for x in missing_R_orderings:
    prediction_vector = np.zeros(len(players))
    for j in x[1]:
        prediction_vector[j] = 1
    characteristic_function[x[0]] = estimator.predict(np.matrix(prediction_vector))


# compute the actual Shapley value
shapleys = shapley.compute_shapley_values(players, characteristic_function)

In [ ]:
print("Had to compute %i instances of the value function " % (len(R_sampled_orderings)))
print(shapleys)

# LOADING IN DATA

In [30]:
sys.path.append("../tfti")
import tfti_infer_generator
import importlib
importlib.reload(tfti_infer_generator)

batch_size=10

infer_fn = tfti_infer_generator.get_infer_fn(
    problem="genomics_binding_deepsea_gm12878",
    model="tfti_transformer",
    hparams_set="tfti_transformer_base",
    hparams="",
    checkpoint_path="/data/akmorrow/tfti/t2t_train/6-64-25/model.ckpt-210001",
    batch_size=batch_size
)


# infer_fn_generator(inputs[i:i+batch_size], targets[i:i+batch_size], keep_mask)

INFO:tensorflow:Unsetting shared_embedding_and_softmax_weights.


[2018-05-02 11:54:14,334] Unsetting shared_embedding_and_softmax_weights.


INFO:tensorflow:Setting T2TModel mode to 'eval'


[2018-05-02 11:54:14,336] Setting T2TModel mode to 'eval'


INFO:tensorflow:Setting hparams.dropout to 0.0


...

INFO:tensorflow:Transforming 'targets' with binary_class_label_modality_512.targets_bottom


[2018-05-02 11:54:14,542] Transforming 'targets' with binary_class_label_modality_512.targets_bottom


INFO:tensorflow:Building model body


[2018-05-02 11:54:14,558] Building model body


ValueError: Dimension 0 in both shapes must be equal, but are 919 and 1000. Shapes are [919,8] and [1000,8]. for 'tfti_transformer/parallel_0_7/tfti_transformer/body/decoder/layer_0/encdec_attention/multihead_attention/dot_product_attention/MatMul' (op: 'BatchMatMul') with input shapes: [919,8,10,64], [1000,8,10,64].

# Load in Generator Data for Validation

In [23]:
batch_size=1

problem_str="genomics_binding_deepsea_gm12878"
model_str="tfti_transformer"
hparams_set_str="tfti_transformer_base"
hparams_str=""
checkpoint_path="/data/akmorrow/tfti/t2t_train/6-64-25/model.ckpt-210001"

infer_fn = tfti_infer.get_infer_fn(
    problem="genomics_binding_deepsea_gm12878",
    model="tfti_transformer",
    hparams_set="tfti_transformer_base",
    hparams="",
    checkpoint_path="/data/akmorrow/tfti/t2t_train/6-64-25/model.ckpt-210001",
    batch_size=batch_size
)



INFO:tensorflow:Unsetting shared_embedding_and_softmax_weights.


[2018-05-02 11:48:49,291] Unsetting shared_embedding_and_softmax_weights.


INFO:tensorflow:Setting T2TModel mode to 'eval'


[2018-05-02 11:48:49,294] Setting T2TModel mode to 'eval'


INFO:tensorflow:Setting hparams.dropout to 0.0


...

[2018-05-02 11:48:49,591] Building model body


INFO:tensorflow:Transforming body output with binary_class_label_modality_512.top


[2018-05-02 11:48:52,975] Transforming body output with binary_class_label_modality_512.top


INFO:tensorflow:Restoring parameters from /data/akmorrow/tfti/t2t_train/6-64-25/model.ckpt-210001


[2018-05-02 11:48:54,275] Restoring parameters from /data/akmorrow/tfti/t2t_train/6-64-25/model.ckpt-210001


In [24]:
tmp_dir = os.path.expanduser("/data/epitome/tmp/")

config = tfti_infer.get_config(problem_str, model_str, hparams_set_str, hparams_str, checkpoint_path)
problem, model, hparams = tfti_infer.get_problem_model_hparams(config)
generator = problem.generator(tmp_dir, is_training=False)
generator_list = list(generator)

INFO:tensorflow:Unsetting shared_embedding_and_softmax_weights.


[2018-05-02 11:48:54,512] Unsetting shared_embedding_and_softmax_weights.


INFO:tensorflow:Setting T2TModel mode to 'eval'


[2018-05-02 11:48:54,514] Setting T2TModel mode to 'eval'


INFO:tensorflow:Setting hparams.dropout to 0.0


...

[2018-05-02 11:48:57,220] Generated 5000 examples.


INFO:tensorflow:Generated 6000 examples.


[2018-05-02 11:48:57,738] Generated 6000 examples.


INFO:tensorflow:Generated 7000 examples.


[2018-05-02 11:48:58,221] Generated 7000 examples.


In [26]:
# reload(tfti_infer)
cell_type_1 = "GM12878"
cell_type_2 = "H1-hESC"


marks = tfti_infer.get_tfs(problem, cell_type_1, cell_type_2)
print(marks)

# get all combs up to 3
power_set = shapley.power_set(marks, depth=1)

# f= open("shapley_values_64_25_gm12878.txt","w+")


inputs  = np.array(list(map(lambda x: x['inputs'], generator_list)))
targets = np.array(list(map(lambda x: x['targets'], generator_list)))

for set_ in power_set[1:2]:

    start = time.time()

    tf.logging.info("Computing average auROC for set %s" % set_)
    # select marks for this run
    selected_marks = [m for m in marks if m in set_]
    
    keep_mask = tfti_infer.get_keep_mask_for_marks(problem, selected_marks, cell_type_1)
    
    # instantiate labels and predictions for this set
    labels_numpy = np.zeros((len(generator_list), len(marks) ))
    predictions_numpy = np.zeros((len(generator_list), len(marks) ))

    for i in range(len(generator_list)):
        response = infer_fn(inputs[i], targets[i], keep_mask)
#         print(response)
#         labels_numpy[i] = response['labels']
#         predictions_numpy[i] = response['predictions']

        if (True):
            end = time.time()
            elapsed = end - start
            start = time.time()
            print(f"Completed {i} iterations in {elapsed} seconds")

#     roc_aucs = []
#     for i in range(len(marks)):
#         # Compute micro-average ROC area for all marks
#         fpr, tpr, _ = roc_curve(labels_numpy[:,i], predictions_numpy[:,i])
#         roc_auc = auc(fpr, tpr)
#         roc_aucs.append(roc_auc)
        
#     average_roc = np.array(roc_aucs).mean()
#     tf.logging.info("Computed average auROC: %s" % (average_roc))
    
#     # save value
#     f.write("%s\t%s\n" % (selected_marks, average_roc))
    
f.close()

INFO:tensorflow:Marks for CellType GM12878: [(204, 'GM12878|ATF2|None'), (205, 'GM12878|ATF3|None'), (207, 'GM12878|BCL11A|None'), (410, 'GM12878|BRCA1|None'), (210, 'GM12878|CEBPB|None'), (412, 'GM12878|CHD1|None'), (413, 'GM12878|CHD2|None'), (127, 'GM12878|CTCF|None'), (53, 'GM12878|DNase|None'), (128, 'GM12878|EZH2|None'), (212, 'GM12878|Egr-1|None'), (216, 'GM12878|GABP|None'), (420, 'GM12878|JunD|None'), (421, 'GM12878|Max|None'), (423, 'GM12878|Mxi1|None'), (223, 'GM12878|NRSF|None'), (428, 'GM12878|Nrf1|None'), (229, 'GM12878|Pol2-4H8|None'), (230, 'GM12878|Pol2|None'), (436, 'GM12878|RFX5|None'), (235, 'GM12878|RXRA|None'), (233, 'GM12878|Rad21|None'), (437, 'GM12878|SIN3A|None'), (236, 'GM12878|SIX5|None'), (237, 'GM12878|SP1|None'), (238, 'GM12878|SRF|None'), (240, 'GM12878|TAF1|None'), (442, 'GM12878|TBP|None'), (241, 'GM12878|TCF12|None'), (243, 'GM12878|USF-1|None'), (444, 'GM12878|USF2|None'), (244, 'GM12878|YY1|None'), (447, 'GM12878|Znf143|None'), (725, 'GM12878|c-Myc|

[2018-05-02 11:49:39,484] Marks for CellType GM12878: [(204, 'GM12878|ATF2|None'), (205, 'GM12878|ATF3|None'), (207, 'GM12878|BCL11A|None'), (410, 'GM12878|BRCA1|None'), (210, 'GM12878|CEBPB|None'), (412, 'GM12878|CHD1|None'), (413, 'GM12878|CHD2|None'), (127, 'GM12878|CTCF|None'), (53, 'GM12878|DNase|None'), (128, 'GM12878|EZH2|None'), (212, 'GM12878|Egr-1|None'), (216, 'GM12878|GABP|None'), (420, 'GM12878|JunD|None'), (421, 'GM12878|Max|None'), (423, 'GM12878|Mxi1|None'), (223, 'GM12878|NRSF|None'), (428, 'GM12878|Nrf1|None'), (229, 'GM12878|Pol2-4H8|None'), (230, 'GM12878|Pol2|None'), (436, 'GM12878|RFX5|None'), (235, 'GM12878|RXRA|None'), (233, 'GM12878|Rad21|None'), (437, 'GM12878|SIN3A|None'), (236, 'GM12878|SIX5|None'), (237, 'GM12878|SP1|None'), (238, 'GM12878|SRF|None'), (240, 'GM12878|TAF1|None'), (442, 'GM12878|TBP|None'), (241, 'GM12878|TCF12|None'), (243, 'GM12878|USF-1|None'), (444, 'GM12878|USF2|None'), (244, 'GM12878|YY1|None'), (447, 'GM12878|Znf143|None'), (725, 'GM12

['ATF2', 'ATF3', 'BCL11A', 'BRCA1', 'CEBPB', 'CHD1', 'CHD2', 'CTCF', 'DNase', 'EZH2', 'Egr-1', 'GABP', 'JunD', 'Max', 'Mxi1', 'NRSF', 'Nrf1', 'Pol2-4H8', 'Pol2', 'RFX5', 'RXRA', 'Rad21', 'SIN3A', 'SIX5', 'SP1', 'SRF', 'TAF1', 'TBP', 'TCF12', 'USF-1', 'USF2', 'YY1', 'Znf143', 'c-Myc', 'p300']
INFO:tensorflow:Computing average auROC for set ['ATF2']


[2018-05-02 11:49:40,404] Computing average auROC for set ['ATF2']


Completed 0 iterations in 0.8520996570587158 seconds
Completed 1 iterations in 0.7890760898590088 seconds
Completed 2 iterations in 0.871934175491333 seconds
Completed 3 iterations in 0.8274929523468018 seconds
Completed 4 iterations in 0.9699907302856445 seconds
Completed 5 iterations in 0.8919696807861328 seconds
Completed 6 iterations in 0.8778502941131592 seconds
Completed 7 iterations in 0.8786303997039795 seconds
Completed 8 iterations in 0.903508186340332 seconds
Completed 9 iterations in 0.878917932510376 seconds
Completed 10 iterations in 0.8727848529815674 seconds
Completed 11 iterations in 0.8905315399169922 seconds
Completed 12 iterations in 0.8719918727874756 seconds
Completed 13 iterations in 0.8504669666290283 seconds
Completed 14 iterations in 0.7817399501800537 seconds
Completed 15 iterations in 0.7955925464630127 seconds
Completed 16 iterations in 0.7937886714935303 seconds
Completed 17 iterations in 0.8312709331512451 seconds
Completed 18 iterations in 0.808183193206

KeyboardInterrupt: 